In [33]:
# %matplotlib qt
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
#from image_slicer import slice

In [34]:
s = hs.load("../data/raw_data/20220325 1529 SI HAADF 3.90 Mx.emd")

Extract 2D array of pixel brightenss from the HAADF image.

In [35]:
img = s[7].data
#type(img)
#print(img, img.shape)

In [36]:
np.save("../data/npy_files/pixel brightness", img)

Extract 3D array of x-ray count of each pixel.
There are three axis: the x and y of the sample surface, and an energy axis of 4096 energy levels. So the data extracted below is already the spectrum map.

In [37]:
spec = s[-1].data
#type(spec)
#print(spec, spec.shape)

In [38]:
np.save("../data/npy_files/pixel spectrum", spec)

If you specify a pixel coordiation for example (0, 0), you obtained an 1D array that records the x-ray signal count along the 4096 energy level.

In [39]:
eg_spec = spec[0][0]
print(eg_spec, len(eg_spec))

[0 0 0 ... 0 0 0] 4096


Then we will be able to define the chemical composition, by examining that spectrum on the signal count (intensity) and the index where that intensity shows up along the energy axis. 
- The only concern is the low signal count here, according to PNNL side. But we could first play with the data format.

## Preprocessing of TEM image data

Numeric values of brightness are loaded based on pre-cropped images.

In [40]:
dom_1 = hs.load("../data/Images/domain 1.png")
dom_2 = hs.load("../data/Images/domain 2.png")
dom_3 = hs.load("../data/Images/domain 3.png")

In [41]:
dom_1_array = np.array(dom_1.data)
dom_2_array = np.array(dom_2.data)
dom_3_array = np.array(dom_3.data)

In [42]:
pix_class = {'domain 1':dom_1_array, 'domain 2': dom_2_array, 'domain 3':dom_3_array}

In [43]:
np.save("../data/npy_files/pixel classification", pix_class)

X-ray spectrums associated with each pixel come from the X-ray map named as "navigator" in the raw data.

Create empty matrices that have the same size of each domain image.

In [44]:
pix_spec = s[-1].data

In [45]:
dom_1_spec = np.empty(dom_1_array.shape, dtype = object)
dom_2_spec = np.empty(dom_2_array.shape, dtype = object)
dom_3_spec = np.empty(dom_3_array.shape, dtype = object)

Iterate over each empty matrix to fill with the 1-d x-ray spectrum at tha pixel.

In [46]:
for i in range(len(dom_1_spec)):
    for j in range(len(dom_1_spec[0])):
        dom_1_spec[(i, j)] = s[-1].data[i, j][30:2000]

In [47]:
for i in range(len(dom_2_spec)):
    for j in range(len(dom_2_spec[0])):
        dom_2_spec[(i, j)] = s[-1].data[i+98, j][30:2000]

In [48]:
for i in range(len(dom_3_spec)):
    for j in range(len(dom_3_spec[0])):
        dom_3_spec[(i, j)] = s[-1].data[i+98+214, j][30:2000]

In [49]:
pix_spec = {'domain 1': dom_1_spec, 'domain 2': dom_2_spec, 'domain 3': dom_3_spec}

In [50]:
np.save('../data/npy_files/xray classification', pix_spec)

## Chip-wise datasets

In [51]:
# use image_slicer to chip HAADF image into 500 tiles.
s[7].save("../data/Images/HAADF image", extension = 'png')

Please answer y or n.
Please answer y or n.
Please answer y or n.


In [52]:
tiles = slice("../data/Images/HAADF image.png", 500)

`.tiff` won't work with slice(). Used `.png`. The HAADF image has been sliced into 506 chips (22x23) with each chip being 22 pixels x 23 pixels.(23 rows x 22 columns). The index in the saved chips possibly represents the coordination (x,y)

In [53]:
# An example to convert chip back to matrix containing grey scale intensity of each pixel. Need to expand to a loop later.
chip = hs.load("../data/Images/HAADF chips_not resized/HAADF image_01_01.png")

In [54]:
chip.data

Array([[ 86, 101,  94,  92,  86,  88,  96, 102, 106, 102,  88,  91, 107,
        106, 127, 115, 110,  94, 104,  94, 126, 139],
       [ 88,  80,  85,  84, 104,  88,  88,  85,  86,  88,  91,  88, 100,
         87,  90,  79,  95, 100, 102, 102, 107, 109],
       [ 77,  82,  79,  99,  96,  86,  82,  73,  86,  88,  88, 101,  93,
        105,  76,  82,  80,  91,  95,  91,  86,  78],
       [ 80,  98, 107,  94,  87,  82,  97, 104, 114,  98,  97,  88,  94,
         87,  97, 114, 107,  83,  92,  92, 115, 117],
       [ 69,  95,  97, 108, 115, 113, 100,  94,  90,  95, 109, 107, 105,
         92,  90,  91, 120, 127, 144, 132, 114, 108],
       [104,  94,  98,  89,  98, 102,  87,  93,  86, 106,  84,  76,  88,
         72,  92,  86,  83,  91,  91,  82,  78,  72],
       [ 92,  79,  85,  79,  97,  84,  79,  89,  92,  96,  83,  94,  82,
         81,  81,  92, 102,  94,  98,  90, 113, 101],
       [ 79,  68,  92,  93,  88,  84,  74,  83,  95, 114, 116,  92,  89,
         98,  92,  86,  89, 106,  97, 

Above is an example matrix consisted of intensity values of each pixel in that chip.

Now we rearrange the numeric matrix forms of tiles to their coordination in the original image.

In [55]:
chip_img = hs.load("../data/Images/HAADF chips_not resized/HAADF image*.png")

In [56]:
chip_array = []
for i in range(len(chip_img)):
    chip_array.append(np.array(chip_img[i].data))

In [57]:
chip_mat = np.empty([23, 22], dtype = object)
k = 0
while k <506:
    for i in range(len(chip_mat)):
        for j in range(len(chip_mat[0])):
            chip_mat[(i, j)] = np.array(chip_array[k])
            k = k+1

In [58]:
np.save("../data/npy_files/chip matrice", chip_mat)

In [59]:
test = np.load("../data/npy_files/chip matrice.npy", allow_pickle = True)

Now chip_mat saves the matrices describing the tiles, the matrices are located at the position of the block matrix as the image chips in the original image. Labeling work could be performed based on the ease of indexing from the file name.

For example, if we want to label image chip `HAADF image_01_01.png` as `domain 1`, we could extract the matrix that represents this chip by

In [60]:
chip_1 = test[8, 21]
print(chip_1)

[[ 70  91 114 108  93  84 118 106  80  87 116 107  99  88 117 118  89  91
  107 115 109 115]
 [ 77  75  84  98 101 102 107  81  69  80  88  86  97 108  95 101  90  86
   75  92  93  92]
 [ 84 108  91  84  86  75  66  77  82 103 105 105  96  81  75  73  90  82
  102  96  96  80]
 [ 85 108 119  96  85 103 131  85  71  92 115 103  96  92 112  97  86  91
  110 111  93  97]
 [ 78  88 134 114  87  76 105 115 104  82 102 101  95  91 110 130 100  97
  110 121 106  97]
 [ 87  87 124 120  88 103 107 122  96  89 110 123  96  97 113 131  87  83
  111 108 106 101]
 [ 84  92 110  88  77  82  91 108  97  89  95 101  80  93 105 118 106 106
  114 103  78  77]
 [101  97 100  93  87  64  73  91  86  92 101  90  82  73  74 111  97  81
  103  96 103  88]
 [ 85  87  95  99 104  85  87  84  63  81  89 104 106 111  87  80  82  81
   93 100  92  99]
 [ 76  77  87 109 101  91 103 105  94  79  90 117 103  90  99  95  91  97
   84 107 102  97]
 [ 75  85  77  92 114 105 101  80  73  72  81  79  92 100 106  88  65 

We can then create a dictionary that incorporate `chip_1` as one value of the key `domain 1`. Just be aware of the indexing in the chip file name and in coding language.

Training and testing sets index from each of the 3 domains.

In [61]:
domain_1_train = [(0,4),(0,15),(0,20,),(1,4),(1,11),(2,0),(2,10),(3,2),(3,7),(3,17),
                  (4,3),(4,5),(4,15),(4,19),(5,1),(5,14),(5,16),(5,18),(5,20),(6,1)]
domain_1_test = [(0,3),(0,12),(0,19,),(1,3),(1,12),(2,1),(2,11),(3,3),(3,4),(3,15),
                 (4,4),(4,6),(4,17),(4,20),(5,3),(5,17),(5,19),(5,21),(5,22),(6,2)]
domain_2_train = [[(7,4),(7,15),(7,20,),(8,4),(8,11),(9,0),(9,10),(10,2),(10,7),(10,17),
                   (11,3),(11,5),(11,15),(11,19),(12,1),(12,14),(12,16),(12,18),(12,20),(13,1)]]
domain_2_test = [(7,3),(7,12),(7,19,),(8,3),(8,12),(9,1),(9,11),(10,3),(10,4),(10,15),
                 (11,4),(11,6),(11,17),(11,20),(12,3),(12,17),(12,19),(12,21),(12,22),(13,2)]
domain_3_train = [(15,4),(15,15),(15,20,),(16,4),(16,11),(17,0),(17,10),(18,2),(18,7),(18,17),
                  (19,3),(19,5),(19,15),(19,19),(20,1),(20,14),(20,16),(20,18),(20,20),(21,1)]
domain_3_test = [(15,3),(15,12),(15,19,),(16,3),(16,12),(17,1),(17,11),(18,3),(18,4),(18,15),
                 (19,4),(19,6),(19,17),(19,20),(20,3),(20,17),(20,19),(20,21),(20,22),(21,2)]

Dictionary creation

In [62]:
dict = {'Domain_1_train' : domain_1_train,'Domain_1_test' : domain_1_test,
        'Domain_2_train' : domain_2_train, 'Domain_2_test' : domain_2_test, 
        'Domain_3_train' : domain_3_train,'Domain_3_test' : domain_3_test,}